In [1]:
import cv2
import joblib
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from skimage.feature import hog

# Cihaz ayarı: GPU varsa GPU, yoksa CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===============================
# 1. CNN Özellik Çıkarıcı Model (DeepCNNFeatureExtractor)
# ===============================
class DeepCNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(DeepCNNFeatureExtractor, self).__init__()
        # Blok 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # 64x64 -> 32x32

        # Blok 2
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3   = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn4   = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)  # 32x32 -> 16x16

        # Blok 3
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn5   = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn6   = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)  # 16x16 -> 8x8

        # Tam bağlantılı katman: 128*8*8'den 256'ya inme
        self.fc = nn.Linear(128 * 8 * 8, 256)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = x.view(x.size(0), -1)
        features = self.fc(x)
        return features

# ===============================
# 2. HOG ve LPG Özellik Çıkarım Fonksiyonları
# ===============================
def extract_hog_features(image):
    image = np.float32(image) / 255.0
    return hog(image, orientations=9, pixels_per_cell=(8,8),
               cells_per_block=(2,2), block_norm='L2-Hys', feature_vector=True)

def extract_lpg_features(image):
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    angle = cv2.phase(grad_x, grad_y, angleInDegrees=True)
    hist = cv2.calcHist([np.uint8(angle)], [0], None, [36], [0, 360])
    return cv2.normalize(hist, hist).flatten()

# ===============================
# 3. Hibrit Özellik Çıkarım Fonksiyonu (CNN + HOG + LPG)
# ===============================
def extract_hybrid_features(image_tensor, cnn_model, original_image):
    cnn_model.eval()
    with torch.no_grad():
        cnn_feat = cnn_model(image_tensor.unsqueeze(0).to(device)).squeeze(0).cpu().numpy()
    hog_feat = extract_hog_features(original_image)
    lpg_feat = extract_lpg_features(original_image)
    return np.concatenate([cnn_feat, hog_feat, lpg_feat])

# ===============================
# 4. Ön işleme ve Model Yüklemesi
# ===============================
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Eğitilmiş SVM modeli, PCA nesnesi ve CNN modelini yükleyin.
best_model = joblib.load("model_svm.pkl")
pca = joblib.load("pca_transformer.pkl")
cnn_extractor = DeepCNNFeatureExtractor().to(device)
cnn_extractor.eval()

# Sınıf isimleri (6 sınıf varsa):
class_names = ["Angry", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# ===============================
# 5. Yüz Tespiti için Haar Cascade (alternatif olarak YOLO kullanılabilir)
# ===============================
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# Eğer YOLO kullanmak isterseniz; YOLO'ya ait ağırlık ve konfigürasyon dosyaları gerekecektir.
# Bu durumda, yüz tespiti için uygun bir YOLO modeli entegre etmeniz gerekecektir.

# ===============================
# 6. Kameradan Canlı Yüz İfadelerinin Tanınması
# ===============================
cap = cv2.VideoCapture(0)  # 0 varsayılan kameradır.

if not cap.isOpened():
    raise Exception("Kamera açılamadı, lütfen kontrol edin.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Frame'i gri tonlamaya çevir
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Yüzleri tespit et
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # Her yüz için döngü
    for (x, y, w, h) in faces:
        # Yüz bölgesini al ve 64x64 boyutuna getir
        face_roi = gray_frame[y:y+h, x:x+w]
        face_roi_resized = cv2.resize(face_roi, (64, 64))

        # PIL formatına dönüştürüp tensöre çevir
        face_pil = Image.fromarray(face_roi_resized)
        image_tensor = transform(face_pil)

        # Hibrit özellik çıkarımı
        hybrid_feat = extract_hybrid_features(image_tensor, cnn_extractor, face_roi_resized)
        X_sample = pca.transform(hybrid_feat.reshape(1, -1))
        predicted_label = best_model.predict(X_sample)[0]
        pred_class_name = class_names[predicted_label] if predicted_label < len(class_names) else str(predicted_label)

        # Yüzün etrafına dikdörtgen çiz ve tahmin yazısını ekle
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, pred_class_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imshow("Canli Yuz Ifade Tanima", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
